In [1]:
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && \
    echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list && \
    sudo apt update && sudo apt install ngrok

deb https://ngrok-agent.s3.amazonaws.com buster main
Get:1 http://packages.cloud.google.com/apt gcsfuse-focal InRelease [1301 B]
Get:2 https://ngrok-agent.s3.amazonaws.com buster InRelease [20.3 kB]          
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease                         
Get:5 https://packages.cloud.google.com/apt cloud-sdk InRelease [6361 B]     
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]       
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]      
Get:8 http://packages.cloud.google.com/apt gcsfuse-focal/main amd64 Packages [17.3 kB]
Get:9 https://packages.cloud.google.com/apt google-fast-socket InRelease [5015 B]0m
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]33m  
Get:11 https://ngrok-agent.s3.amazonaws.com buster/main amd64 Packages [3407 B]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updat

In [2]:
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
ngrok_authtoken = user_secrets.get_secret("NGROK_AUTHTOKEN")

In [3]:
from subprocess import Popen
import time

Popen(f"ngrok config add-authtoken {ngrok_authtoken}", shell=True)
time.sleep(1)

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [4]:
Popen("ngrok http 6006", shell=True)
time.sleep(1)

In [5]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print('Tensorboard URL:', json.load(sys.stdin)['tunnels'][0]['public_url'])"

Tensorboard URL: https://d77f-34-82-74-17.ngrok-free.app


In [6]:
Popen("tensorboard --logdir ./lightning_logs/ --host 0.0.0.0 --port 6006", shell=True)

<Popen: returncode: None args: 'tensorboard --logdir ./lightning_logs/ --hos...>

In [7]:
!ps aux | grep tensorboard

root         695  0.0  0.0   2892  1060 ?        S    15:01   0:00 /bin/sh -c tensorboard --logdir ./lightning_logs/ --host 0.0.0.0 --port 6006
root         696 86.0  0.1 224640 47756 ?        Rl   15:01   0:00 /opt/conda/bin/python3 /opt/conda/bin/tensorboard --logdir ./lightning_logs/ --host 0.0.0.0 --port 6006
root         697 89.0  0.0   4780  3172 pts/0    Ss+  15:01   0:00 /bin/bash -c ps aux | grep tensorboard
root         704  0.0  0.0    648     8 pts/0    R+   15:01   0:00 grep tensorboard


In [8]:
# %load_ext tensorboard
# %tensorboard --logdir ./lightning_logs/ --host 0.0.0.0 --port 6006

In [9]:
!pip install --no-deps d2l

  Obtaining dependency information for d2l from https://files.pythonhosted.org/packages/8b/39/418ef003ed7ec0f2a071e24ec3f58c7b1f179ef44bec5224dcca276876e3/d2l-1.0.3-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 4.1 MB/s eta 0:00:00


In [10]:
import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.tuner import Tuner
from torch import nn
from torch.utils.data import DataLoader, random_split
from torchmetrics import Accuracy
from torchvision.datasets import FashionMNIST
from torchvision.transforms import Compose, Resize, ToTensor


torch.set_float32_matmul_precision("medium")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [11]:
conv_arch = {
    "11": ((1, 64), (1, 128), (2, 256), (2, 512), (2, 512)),
    "13": ((2, 64), (2, 128), (2, 256), (2, 512), (2, 512)),
    "16": ((2, 64), (2, 128), (3, 256), (3, 512), (3, 512)),
    "19": ((2, 64), (2, 128), (4, 256), (4, 512), (4, 512)),
}

ratio = 4
small_conv_arch = {
    key: [(pair[0], pair[1] // ratio) for pair in value] for key, value in conv_arch.items()
}

In [12]:
class FashionMNISTDataModel(pl.LightningDataModule):
    def __init__(self, batch_size=128, data_dir="../data", num_workers=4):
        super().__init__()
        self.batch_size = batch_size
        self.data_dir = data_dir
        self.num_workers = num_workers
        self.trans = Compose([ToTensor(), Resize(224, antialias=True)])

    def prepare_data(self):
        FashionMNIST(root=self.data_dir, download=True)

    def setup(self, stage=None):
        if stage == "fit" or stage is None:
            self.train_data, self.val_data = random_split(
                FashionMNIST(root=self.data_dir, train=True, transform=self.trans), [0.8, 0.2]
            )
        elif stage == "test" or stage is None:
            self.test_data = FashionMNIST(root=self.data_dir, train=False, transform=self.trans)

    def train_dataloader(self):
        return DataLoader(
            self.train_data,
            self.batch_size,
            num_workers=self.num_workers,
            pin_memory=True,
            shuffle=True,
        )

    def val_dataloader(self):
        return DataLoader(
            self.val_data,
            self.batch_size,
            num_workers=self.num_workers,
            pin_memory=True,
            shuffle=False,
        )

    def test_dataloader(self):
        return DataLoader(
            self.test_data,
            self.batch_size,
            num_workers=self.num_workers,
            pin_memory=True,
            shuffle=False,
        )

In [13]:
def vgg_block(num_convs, in_channels, out_channels):
    layers = []
    for _ in range(num_convs):
        layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1))
        layers.append(nn.ReLU())
        in_channels = out_channels
    layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
    return nn.Sequential(*layers)

In [14]:
class Vgg(pl.LightningModule):
    def __init__(self, conv_arch, lr=0.05, weight_decay=0):
        super().__init__()
        # self.lr = lr
        # self.weight_decay = weight_decay
        self.save_hyperparameters()

        conv_blks = []
        in_channels = 1
        for num_convs, out_channels in conv_arch:
            conv_blks.append(vgg_block(num_convs, in_channels, out_channels))
            in_channels = out_channels

        self.features = nn.Sequential(*conv_blks)
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(out_channels * 7 * 7, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 10),
        )

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, mode="fan_in", nonlinearity="relu")
                nn.init.constant_(m.bias, 0)

        self.loss = nn.CrossEntropyLoss()
        self.train_acc = Accuracy(task="multiclass", num_classes=10, average="micro")
        self.val_acc = Accuracy(task="multiclass", num_classes=10, average="micro")
        self.test_acc = Accuracy(task="multiclass", num_classes=10, average="micro")

    def forward(self, X):
        X = self.features(X)
        X = self.classifier(X)
        return X

    def training_step(self, batch):
        X, y = batch
        y_hat = self(X)
        loss = self.loss(y_hat, y)
        acc = self.train_acc(y_hat, y)
        metrics = {"train_loss": loss, "train_acc": acc}
        self.log_dict(metrics, prog_bar=True)
        return loss

    def validation_step(self, batch):
        X, y = batch
        y_hat = self(X)
        loss = self.loss(y_hat, y)
        acc = self.val_acc(y_hat, y)
        metrics = {"val_loss": loss, "val_acc": acc}
        self.log_dict(metrics, prog_bar=True)
        return loss

    def test_step(self, batch):
        X, y = batch
        y_hat = self(X)
        loss = self.loss(y_hat, y)
        acc = self.test_acc(y_hat, y)
        metrics = {"test_loss": loss, "test_acc": acc}
        self.log_dict(metrics, prog_bar=True)
        return loss

    def on_test_end(self):
        self.logger.log_hyperparams(
            self.hparams,
            {
                "hp/train_acc": self.train_acc.compute(),
                "hp/val_acc": self.val_acc.compute(),
                "hp/test_acc": self.test_acc.compute(),
            },
        )

    def configure_optimizers(self):
        return torch.optim.SGD(
            self.parameters(), lr=self.hparams.lr, weight_decay=self.hparams.weight_decay
        )

In [15]:
def cks(name):
    ck_train_loss = ModelCheckpoint(
        monitor="train_loss", filename=f"{name}" + "-{epoch:02d}-{train_loss:.2f}"
    )
    ck_train_acc = ModelCheckpoint(
        monitor="train_acc", mode="max", filename=f"{name}" + "-{epoch:02d}-{val_acc:.2f}"
    )
    ck_val_loss = ModelCheckpoint(
        monitor="val_loss", filename=f"{name}" + "-{epoch:02d}-{val_loss:.2f}"
    )
    ck_val_acc = ModelCheckpoint(
        monitor="val_acc", mode="max", filename=f"{name}" + "-{epoch:02d}-{val_acc:.2f}"
    )
    return [ck_train_loss, ck_train_acc, ck_val_loss, ck_val_acc]

In [16]:
data = FashionMNISTDataModel(batch_size=128)
data.prepare_data()

100%|██████████| 26421880/26421880 [00:01<00:00, 13225613.57it/s]


Extracting ../data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ../data/FashionMNIST/raw




NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784



TensorBoard 2.13.0 at http://0.0.0.0:6006/ (Press CTRL+C to quit)
100%|██████████| 29515/29515 [00:00<00:00, 209163.85it/s]


Extracting ../data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ../data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3903427.67it/s]


Extracting ../data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ../data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 10763846.96it/s]

Extracting ../data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/FashionMNIST/raw



训练 VGG-11

In [17]:
model = Vgg(small_conv_arch["11"])
logger = TensorBoardLogger(".", default_hp_metric=False)
trainer = pl.Trainer(max_epochs=10, logger=logger, callbacks=cks("vgg11"))

In [18]:
%%time
trainer.fit(model, data)

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

CPU times: user 7min 43s, sys: 57.4 s, total: 8min 41s
Wall time: 7min 54s


In [19]:
trainer.test(model, data)

Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.9161999821662903     │
│         test_loss         │    0.23104605078697205    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.23104605078697205, 'test_acc': 0.9161999821662903}]

In [20]:
trainer.save_checkpoint("vgg11_epoch=10.ckpt")

训练 VGG-13

In [21]:
checkpoint = torch.load("vgg11_epoch=10.ckpt")
model = Vgg(small_conv_arch["13"])
model.load_state_dict(checkpoint["state_dict"], strict=False)
model.hparams.lr = checkpoint["hyper_parameters"]["lr"]
model.hparams.weight_decay = checkpoint["hyper_parameters"]["weight_decay"]
logger = TensorBoardLogger(".", default_hp_metric=False)
trainer = pl.Trainer(max_epochs=10, logger=logger, callbacks=cks("vgg13"))

In [22]:
%%time
trainer.fit(model, data)

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

CPU times: user 11min 42s, sys: 52 s, total: 12min 34s
Wall time: 11min 33s


In [23]:
trainer.test(model, data)

Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │     0.926800012588501     │
│         test_loss         │    0.23583251237869263    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.23583251237869263, 'test_acc': 0.926800012588501}]

In [24]:
trainer.save_checkpoint("vgg13_epoch=10.ckpt")

训练 VGG-16

In [25]:
checkpoint = torch.load("vgg13_epoch=10.ckpt")
model = Vgg(small_conv_arch["16"])
model.load_state_dict(checkpoint["state_dict"], strict=False)
model.hparams.lr = checkpoint["hyper_parameters"]["lr"]
model.hparams.weight_decay = checkpoint["hyper_parameters"]["weight_decay"]
logger = TensorBoardLogger(".", default_hp_metric=False)
trainer = pl.Trainer(max_epochs=10, logger=logger, callbacks=cks("vgg16"))

In [26]:
%%time
trainer.fit(model, data)

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

CPU times: user 13min, sys: 52.8 s, total: 13min 53s
Wall time: 12min 54s


In [27]:
trainer.test(model, data)

Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.9244999885559082     │
│         test_loss         │    0.2127581536769867     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.2127581536769867, 'test_acc': 0.9244999885559082}]

In [28]:
trainer.save_checkpoint("vgg16_epoch=10.ckpt")

训练 VGG-19

In [29]:
checkpoint = torch.load("vgg16_epoch=10.ckpt")
model = Vgg(small_conv_arch["19"])
model.load_state_dict(checkpoint["state_dict"], strict=False)
model.hparams.lr = checkpoint["hyper_parameters"]["lr"]
model.hparams.weight_decay = checkpoint["hyper_parameters"]["weight_decay"]
logger = TensorBoardLogger(".", default_hp_metric=False)
trainer = pl.Trainer(max_epochs=10, logger=logger, callbacks=cks("vgg19"))

In [30]:
%%time
trainer.fit(model, data)

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

CPU times: user 14min 20s, sys: 54.9 s, total: 15min 15s
Wall time: 14min 15s


In [31]:
trainer.test(model, data)

Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.9315999746322632     │
│         test_loss         │    0.20251916348934174    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.20251916348934174, 'test_acc': 0.9315999746322632}]

In [32]:
trainer.save_checkpoint("vgg19_epoch=10.ckpt")

In [33]:
from IPython.display import FileLink

In [34]:
FileLink("vgg11_epoch=10.ckpt")

/kaggle/working/vgg11_epoch=10.ckpt

In [35]:
FileLink("vgg13_epoch=10.ckpt")

/kaggle/working/vgg13_epoch=10.ckpt

In [36]:
FileLink("vgg16_epoch=10.ckpt")

/kaggle/working/vgg16_epoch=10.ckpt

In [37]:
FileLink("vgg19_epoch=10.ckpt")

/kaggle/working/vgg19_epoch=10.ckpt

In [39]:
model = Vgg.load_from_checkpoint("vgg19_epoch=10.ckpt")
logger = TensorBoardLogger(".", default_hp_metric=False)
trainer = pl.Trainer(max_epochs=10, logger=logger, callbacks=cks("vgg19"))

In [40]:
%%time
trainer.fit(model, data)

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

CPU times: user 14min 18s, sys: 50.3 s, total: 15min 8s
Wall time: 14min 7s


In [41]:
trainer.test(model, data)

Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.9289000034332275     │
│         test_loss         │    0.2666994631290436     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.2666994631290436, 'test_acc': 0.9289000034332275}]

In [42]:
trainer.save_checkpoint("vgg19_epoch=20.ckpt")

In [43]:
model = Vgg.load_from_checkpoint("vgg16_epoch=10.ckpt")
logger = TensorBoardLogger(".", default_hp_metric=False)
trainer = pl.Trainer(max_epochs=10, logger=logger, callbacks=cks("vgg16"))

In [44]:
%%time
trainer.fit(model, data)

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

CPU times: user 13min, sys: 52.8 s, total: 13min 53s
Wall time: 12min 54s


In [45]:
trainer.test(model, data)

Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.9332000017166138     │
│         test_loss         │    0.25012871623039246    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.25012871623039246, 'test_acc': 0.9332000017166138}]

In [46]:
trainer.save_checkpoint("vgg16_epoch=20.ckpt")

In [47]:
model = Vgg.load_from_checkpoint("vgg13_epoch=10.ckpt")
logger = TensorBoardLogger(".", default_hp_metric=False)
trainer = pl.Trainer(max_epochs=10, logger=logger, callbacks=cks("vgg13"))

In [48]:
%%time
trainer.fit(model, data)

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

CPU times: user 11min 39s, sys: 51.5 s, total: 12min 31s
Wall time: 11min 31s


In [49]:
trainer.test(model, data)

Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.9312999844551086     │
│         test_loss         │    0.2735268175601959     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.2735268175601959, 'test_acc': 0.9312999844551086}]

In [50]:
trainer.save_checkpoint("vgg13_epoch=20.ckpt")

In [51]:
model = Vgg.load_from_checkpoint("vgg11_epoch=10.ckpt")
logger = TensorBoardLogger(".", default_hp_metric=False)
trainer = pl.Trainer(max_epochs=10, logger=logger, callbacks=cks("vgg11"))

In [52]:
%%time
trainer.fit(model, data)

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

CPU times: user 7min 31s, sys: 51 s, total: 8min 22s
Wall time: 7min 31s


In [53]:
trainer.test(model, data)

Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.9296000003814697     │
│         test_loss         │    0.22261880338191986    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.22261880338191986, 'test_acc': 0.9296000003814697}]

In [54]:
trainer.save_checkpoint("vgg11_epoch=20.ckpt")